In [ ]:
from db_actions import DatabaseActions as db

data = db.get_all()

for i in range(len(data[0])):
    print(i, data[0][i])

In [ ]:
texts = []
for r in data:
    texts.append(r[4])

print(texts[0], len(texts))



In [ ]:
def quicksort_by_length(arr):
    if len(arr) <= 1:
        return arr

    pivot = arr[len(arr) // 2]
    shorter = [text for text in arr if len(text) < len(pivot)]
    equal = [text for text in arr if len(text) == len(pivot)]
    longer = [text for text in arr if len(text) > len(pivot)]

    return quicksort_by_length(shorter) + equal + quicksort_by_length(longer)

# Example usage:
sorted_texts = quicksort_by_length(texts)
for text in sorted_texts:
    print(text, end='\n' + '~' * 80 + '\n')

In [ ]:
from get_config import write_list

write_list(sorted_texts, "./data/sorted_texts.bin")

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def levenshtein_distance(word1, word2):
    m = len(word1)
    n = len(word2)

    # Создаем матрицу размером (m+1) x (n+1)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Инициализируем первую строку и первый столбец
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Заполняем остальные ячейки матрицы
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1]) + 1

    return dp[m][n]


def sentence_similarity(sentence1, sentence2):
    words1 = sentence1.split()
    words2 = sentence2.split()
    similarity = 0.0

    for word1 in words1:
        word_similarity = 0.0
        for word2 in words2:
            word_distance = levenshtein_distance(word1, word2)
            word_similarity = max(word_similarity, 1.0 - (word_distance / max(len(word1), len(word2))))
        similarity += word_similarity

    similarity /= len(words1)

    return similarity



# Example usage
words1 = "apple banana orange"
words2 = "banana orange kiwi"
similarity = sentence_similarity(words1, words2)
print(similarity)


In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')

from nltk import word_tokenize
def tokenizer(x):
    return ( w for w in word_tokenize(x) if len(w) >3)

def sentence_similarity(sentence1, sentence2):
    corpus = [sentence1, sentence2]

    # Преобразуем предложения в матрицу TF-IDF
    vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Вычисляем косинусное расстояние между предложениями
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

    return similarity


def word_similarity(word1, word2):
    corpus = [word1, word2]

    # Преобразуем слова в матрицу TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Вычисляем косинусное расстояние между словами
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

    return similarity


In [ ]:
def find_max_similarity(text, texts):
    dif = 0.1
    c = 0
    length = len(text)
    similarity = 0.0
    for other_text in texts:
        other_length = len(other_text)
        if other_length >= length * (1 + dif):
            break
        if other_length >= length * (1 - dif):
            c += 1
            current_similarity = sentence_similarity(text, other_text)
            if current_similarity > similarity and current_similarity <= (1 - 0.1 ** 2):
                    print(current_similarity, other_text)
                    print("~" * 80)
                    similarity = current_similarity
            # similarity = max(similarity, current_similarity)
    print(c)
    return similarity



In [ ]:
txt = '''Большой современный ewfwefwfwefwf ресторан в Москве. Отличное место. Красивый интерьер. 
Большой выбор блюд. Хороший сервис, официант всегда может помочь с выбором блюд. 
feefef кутабы с efefef, зеленью и тыквой.  wefweff всегда на высоте. Вот только ffff по сравнению с другими ресторанами уступает, маленькая порция, с рыбой не советую совсем слишком зажарена(буквально три маленьких кусочка)
Отмечали здесь не один большой праздник, всегда все довольны. Все ваши капризы как wefwefw за ваш счет. 
Летом efefef efefef с видом на старый Арбат'''

print(find_max_similarity(txt, sorted_texts))